In [ ]:
import pandas as pd

In [ ]:
#输入DF、文件名称，生成EXCEL表格
def generate_excel(df, f_name):
    if (df.shape[0] == 0):
        print ('没有数据产出。')
    else:
        if os.path.exists(f_name):
            os.remove(f_name)
        df.to_excel(f_name, index=False)
        print(f_name, '文件已生成。')

#根据DF的某个列生成文件夹
def generate_folder(df, field):
    for f in list(df[field].unique()):
        print (f)
        if os.path.exists(f):
#             os.remove(f) #会报错：remove用于删除文件
            os.rmdir(f) #rmdir用于删除文件夹
        os.mkdir(f)
    print('Done')


#将文件列表解析成一个DF
def concat_files(files):
    df = pd.DataFrame()
    for f in files:
        if re.search('.xlsx', f) != None:
            data = pd.read_excel(f)
        if re.search('.csv', f) != None:
            data = pd.read_csv(f)
        df = pd.concat([df, data], axis=0, ignore_index=True)
    return df

#获得【标准】网址名
def get_clean_url(s):
    s = s.lower()
    s = re.sub("\s", "", s)
    s = re.sub("/", "", s)
    match = re.search('www.', s)
    site = s
    if match != None:
        start = match.span()[1]
        site = s[start:]
    return site

#获得已经关停的商户
def get_down_mers():
    down_mers = pd.read_excel('资料汇总.xlsx', sheet_name='关停商户')
    down_mers = down_mers.applymap(str)
    down_mers['商户号'] = down_mers['商户号'].map(lambda x: x.replace('.0', ''))
    return down_mers

#获得已经关停的网站
def get_down_urls():
    down_urls = pd.read_excel('资料汇总.xlsx', sheet_name='关停网站')
    down_urls = down_urls.applymap(str)
    down_urls['商户号'] = down_urls['商户号'].map(lambda x: x.replace('.0', ''))
    return down_urls

#获得商户详情，如AM/BD
def get_mer_info():
    mer_info = pd.read_excel('资料汇总.xlsx', sheet_name='AM分配')
    mer_info = mer_info.applymap(str)
    mer_info['商户号'] = mer_info['商户号'].map(lambda x: x.replace('.0', ''))
    return mer_info

#查看商户是否分配AM
def check_am(df):
    no_ams = df[df['AM'].isnull()]
    nan_ams = df[df['AM']=='nan']
    if((no_ams.shape[0] != 0) or (nan_ams.shape[0] !=0)):
        print('如下商户还未分配AM：')
        for mer in list(no_ams['商户号'].unique()):
            print (mer)
        for mer in list(nan_ams['商户号'].unique()):
            print (mer)
    else:
        print('AM都已分配妥当。')

#循环输出某一列的唯一值，如输出所有的AM/BD/商户号等
def iterate_col(df, col):
    items = df[col].unique()
    for i in list(items):
        print('      ',i)

#根据某一列的唯一值分割DF,并返回DF列表
def split_dfs(df, col):
    dfs = []
    items = df[col].unique()
    for i in list(items):
#         print(i)
        sub_df = df[df[col] == i]
        if (sub_df.shape[0] != 0):
            dfs.append(sub_df)
    return dfs

In [ ]:
# 网站复审 2022, 2, 17 Thur
# 0）去重
# 1)删除停用的商户
# 2）商户停用的网站
# 3）匹配AM/BD
sites = pd.read_excel('网址匹配商户号20200207.xlsx')
# 0）去重
sites = sites.applymap(str)
sites['URL'] = sites['URL'].map(lambda x: x.lower())
sites.drop_duplicates('URL', inplace=True)
print('去重后：', sites.shape[0])
# sites[sites['URL'].duplicated(keep=False)].sort_values('URL')

# 1)删除停用的商户
down_mers = get_down_mers()
sites[sites['商户号'].isin(set(down_mers['商户号']))] #查看是否存在关停商户
sites.drop(sites[sites['商户号'].isin(set(down_mers['商户号']))].index, inplace=True)
print('删除关停商户相关的记录后：', sites.shape[0])

# 2）商户停用的网站
down_urls = get_down_urls()
sites.drop(sites[sites['URL'].isin(set(down_urls['URL']))].index, inplace=True)
print('删除关停网站相关的记录后：', sites.shape[0])

# 3）匹配AM/BD
mer_info = get_mer_info()
urls = sites.merge(mer_info[['AM', 'BD', '商户号']], on='商户号', how='left')
check_am(urls)

In [ ]:
sample = urls.head(20)
urls_4am = split_dfs(sample, 'AM')
for df in urls_4am:
    am = df.AM.unique()
    print ('To', am[0])
    ms = split_dfs(df, '商户号')
    for m in ms:
        m_no = m['商户号'].unique()
        print('   To', m_no[0])
        iterate_col(m, 'URL' )    